# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:
    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

2. Procesamiento de las Consultas:
    * Definir las consultas proporcionadas.
    * Extraer las palabras clave de cada consulta.

3. Cálculo de Relevancia:
    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

4. Ranking de Documentos:
    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
    * Mostrar el ranking de documentos para cada consulta.

In [191]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [192]:
corpus = parse_corpus('/content/03ranking_corpus.xml')
corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

In [193]:
print(len(corpus))

30


In [194]:
docs = []
for doc_id in corpus:
    docs.append(corpus[doc_id]['title'])

print(len(docs))
print(docs)

30
['El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.', 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.', 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.', 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.', 'Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés.', 'La influencia de la cultura universitaria en los hábitos saludables y el bienestar de los estudiantes.', 'La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.', 'La influencia del apoyo familiar y social en el bienestar emocional de los estudiantes universitarios.', 'El impacto de los hábitos de lectura en el desarrollo cognitivo en jóvenes y adultos.', 'Avances en la tecnología para la medicina preventiva y su aplicación en jóvenes.', 'Impacto de la práct

In [195]:
vocab = []
for doc_id in corpus:
    vocab.extend(corpus[doc_id]['keywords'])
vocab = set(vocab)

print(len(vocab))
print(vocab)

115
{'social', 'académica', 'nutrición', 'creatividad', 'meditación', 'sociales', 'beneficios', 'mindfulness', 'extracurriculares', 'universitaria', 'grupos', 'eventos', 'deportes', 'academia', 'servicios', 'impacto', 'médica', 'reducción', 'de', 'relaciones', 'estrés', 'equilibrio', 'universitarios', 'tecnológicos', 'universitario', 'exámenes', 'artísticas', 'programas', 'estudiantil', 'actividades', 'comunidades', 'análisis', 'desarrollo', 'estudio', 'tecnologías', 'avances', 'agotamiento', 'sostenible', 'compartidas', 'salud', 'lectura', 'prevención', 'espacios', 'tecnología', 'interacción', 'bibliotecas', 'talleres', 'comparación', 'psicológico', 'desempeño', 'universidad', 'universitarias', 'competencias', 'becas', 'amistad', 'tecnológicas', 'del', 'tendencias', 'calidad', 'bicicletas', 'familiar', 'en', 'ejercicio', 'saludables', 'cognitivo', 'edificios', 'educación', 'estudiantes', 'telemedicina', 'transporte', 'laboratorios', 'sostenibilidad', 'rendimiento', 'hábitos', 'activid

# Calculo de la matriz tf-idf

In [196]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#Funcion para obtener la matriz tf-idf usando sklearn

def tfidf_transform(vocab, matrix):
  count_vectorizer = CountVectorizer(vocabulary=vocab)
  tf_matrix = count_vectorizer.fit_transform(matrix)
  tfidf_transformer = TfidfTransformer()
  tfidf_matrix = tfidf_transformer.fit_transform(tf_matrix).toarray()
  return tfidf_matrix

In [197]:
#Obtener la matriz tf-idf de los docuemntos del corpus
tfidf_matrix = tfidf_transform(vocab, docs)
print(tfidf_matrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.34326165 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.27033245 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [198]:
#Obtener la matriz tf-idf de las queries
queries_matrix = []
for i, val in queries.items():
  queries_matrix.append(queries[i])

qr_tfidf_matrix = tfidf_transform(vocab, queries_matrix)
print(qr_tfidf_matrix)

[[0.         0.         0.30251368 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.60502736 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.17866945 0.         0.         0.         0.
  0.23006945 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.30251368
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.30251368 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.30251368 0.         0.         0.30251368
  0.         0.         0.         0.         0.         0.
  0.         0.         

# Calculo de la similitud coseno

In [199]:
#Calculo de la similitud coseno
from math import sqrt
def similitudCoseno(v1, v2):
    similitud = []

    for i in range(0, len(v1)):
        producto = 0
        sum1 = 0
        sum2 = 0
        for j in range(0, len(v1[0])):
            producto += v1[i][j] * v2[j]
            sum1 += v1[i][j] ** 2
            sum2 +=  v2[j] ** 2

        sim = producto/(sqrt(sum1) * sqrt(sum2))
        similitud.append(round(sim,3))
    return similitud

In [200]:
cosenoMatrix = []

for i in range(0,len(qr_tfidf_matrix)):
  cosenoMatrix.append(similitudCoseno(tfidf_matrix, qr_tfidf_matrix[i]))

print(cosenoMatrix)

[[0.436, 0.558, 0.269, 0.227, 0.111, 0.263, 0.541, 0.254, 0.291, 0.049, 0.531, 0.259, 0.388, 0.428, 0.233, 0.273, 0.118, 0.141, 0.333, 0.332, 0.302, 0.123, 0.477, 0.186, 0.025, 0.324, 0.232, 0.102, 0.401, 0.022], [0.0, 0.039, 0.297, 0.035, 0.175, 0.15, 0.067, 0.292, 0.056, 0.065, 0.062, 0.062, 0.167, 0.035, 0.595, 0.029, 0.17, 0.13, 0.032, 0.056, 0.031, 0.034, 0.159, 0.037, 0.312, 0.03, 0.153, 0.125, 0.032, 0.029], [0.0, 0.142, 0.125, 0.293, 0.237, 0.122, 0.085, 0.123, 0.071, 0.083, 0.333, 0.153, 0.289, 0.044, 0.078, 0.037, 0.041, 0.106, 0.306, 0.138, 0.04, 0.043, 0.09, 0.348, 0.042, 0.11, 0.108, 0.206, 0.041, 0.037]]


# Calculo de la similitud Jaccard

In [201]:
# Calculo de la similitud Jaccard

def similitudJaccard(v1, v2):
    jaccardMatrix = []
    for i in range(len(v1)):
        similitudMatrix = []
        for j in range(len(v2)):
            intersection = 0
            union = 0
            for k in range(len(v1[0])):
                if v1[i][k] and v2[j][k]:
                    intersection += 1
                if v1[i][k] or v2[j][k]:
                    union += 1
            similitud = intersection/union
            similitudMatrix.append(round(similitud, 3))
        jaccardMatrix.append(similitudMatrix)
    return jaccardMatrix

In [208]:
jaccardMatrix = similitudJaccard(qr_tfidf_matrix,tfidf_matrix)
print(jaccardMatrix)

[[0.182, 0.6, 0.214, 0.154, 0.143, 0.214, 0.5, 0.267, 0.2, 0.071, 0.462, 0.286, 0.375, 0.417, 0.133, 0.143, 0.143, 0.25, 0.385, 0.267, 0.214, 0.143, 0.333, 0.214, 0.067, 0.214, 0.2, 0.071, 0.308, 0.067], [0.0, 0.077, 0.25, 0.083, 0.167, 0.154, 0.067, 0.214, 0.067, 0.083, 0.062, 0.067, 0.111, 0.071, 0.364, 0.077, 0.167, 0.182, 0.067, 0.062, 0.071, 0.077, 0.077, 0.071, 0.273, 0.071, 0.143, 0.083, 0.071, 0.077], [0.0, 0.2, 0.182, 0.222, 0.2, 0.182, 0.077, 0.154, 0.077, 0.1, 0.25, 0.167, 0.2, 0.083, 0.083, 0.091, 0.091, 0.222, 0.273, 0.154, 0.083, 0.091, 0.091, 0.3, 0.091, 0.182, 0.167, 0.1, 0.083, 0.091]]


# Ranking por similitud Coseno

In [203]:
ranking = []

for i in range(0,len(cosenoMatrix)):
  indices = {}
  for j in range(0,len(cosenoMatrix[i])):
    indiceqr = j
    indices[indiceqr] = cosenoMatrix[i][j]
  ranking.append(dict(sorted(indices.items(), key=lambda item: item[1], reverse=True)))

print(ranking)


[{1: 0.558, 6: 0.541, 10: 0.531, 22: 0.477, 0: 0.436, 13: 0.428, 28: 0.401, 12: 0.388, 18: 0.333, 19: 0.332, 25: 0.324, 20: 0.302, 8: 0.291, 15: 0.273, 2: 0.269, 5: 0.263, 11: 0.259, 7: 0.254, 14: 0.233, 26: 0.232, 3: 0.227, 23: 0.186, 17: 0.141, 21: 0.123, 16: 0.118, 4: 0.111, 27: 0.102, 9: 0.049, 24: 0.025, 29: 0.022}, {14: 0.595, 24: 0.312, 2: 0.297, 7: 0.292, 4: 0.175, 16: 0.17, 12: 0.167, 22: 0.159, 26: 0.153, 5: 0.15, 17: 0.13, 27: 0.125, 6: 0.067, 9: 0.065, 10: 0.062, 11: 0.062, 8: 0.056, 19: 0.056, 1: 0.039, 23: 0.037, 3: 0.035, 13: 0.035, 21: 0.034, 18: 0.032, 28: 0.032, 20: 0.031, 25: 0.03, 15: 0.029, 29: 0.029, 0: 0.0}, {23: 0.348, 10: 0.333, 18: 0.306, 3: 0.293, 12: 0.289, 4: 0.237, 27: 0.206, 11: 0.153, 1: 0.142, 19: 0.138, 2: 0.125, 7: 0.123, 5: 0.122, 25: 0.11, 26: 0.108, 17: 0.106, 22: 0.09, 6: 0.085, 9: 0.083, 14: 0.078, 8: 0.071, 13: 0.044, 21: 0.043, 24: 0.042, 16: 0.041, 28: 0.041, 20: 0.04, 15: 0.037, 29: 0.037, 0: 0.0}]


In [204]:
for i in range(len(ranking)):
  print(f'Query {i+1}: {queries[i+1]}\n')
  for j in range(len(ranking[i])):
    documento = list(ranking[i].keys())[j] #Para obtener los indices para buscar los documentos
    similitud = list(ranking[i].values())[j]
    print(f'Similitud: {similitud} | Documento {documento + 1}:{docs[documento]}')
  print('------------------------------------------------------------')



Query 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios

Similitud: 0.558 | Documento 2:Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
Similitud: 0.541 | Documento 7:La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
Similitud: 0.531 | Documento 11:Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.
Similitud: 0.477 | Documento 23:Cómo la carga de trabajo académico afecta la salud emocional de los estudiantes y qué pueden hacer las universidades para ayudar.
Similitud: 0.436 | Documento 1:El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
Similitud: 0.428 | Documento 14:Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
Similitud: 0.401 | Documento 29:Análisis de los hábitos de sueño y su relación con el rendimie

# Ranking por Similitud Jaccard

In [205]:
ranking = []

for i in range(0,len(jaccardMatrix)):
  indices = {}
  for j in range(0,len(jaccardMatrix[i])):
    indiceqr = j
    indices[indiceqr] = jaccardMatrix[i][j]
  ranking.append(dict(sorted(indices.items(), key=lambda item: item[1], reverse=True)))

print(ranking)


[{1: 0.6, 6: 0.5, 10: 0.462, 13: 0.417, 18: 0.385, 12: 0.375, 22: 0.333, 28: 0.308, 11: 0.286, 7: 0.267, 19: 0.267, 17: 0.25, 2: 0.214, 5: 0.214, 20: 0.214, 23: 0.214, 25: 0.214, 8: 0.2, 26: 0.2, 0: 0.182, 3: 0.154, 4: 0.143, 15: 0.143, 16: 0.143, 21: 0.143, 14: 0.133, 9: 0.071, 27: 0.071, 24: 0.067, 29: 0.067}, {14: 0.364, 24: 0.273, 2: 0.25, 7: 0.214, 17: 0.182, 4: 0.167, 16: 0.167, 5: 0.154, 26: 0.143, 12: 0.111, 3: 0.083, 9: 0.083, 27: 0.083, 1: 0.077, 15: 0.077, 21: 0.077, 22: 0.077, 29: 0.077, 13: 0.071, 20: 0.071, 23: 0.071, 25: 0.071, 28: 0.071, 6: 0.067, 8: 0.067, 11: 0.067, 18: 0.067, 10: 0.062, 19: 0.062, 0: 0.0}, {23: 0.3, 18: 0.273, 10: 0.25, 3: 0.222, 17: 0.222, 1: 0.2, 4: 0.2, 12: 0.2, 2: 0.182, 5: 0.182, 25: 0.182, 11: 0.167, 26: 0.167, 7: 0.154, 19: 0.154, 9: 0.1, 27: 0.1, 15: 0.091, 16: 0.091, 21: 0.091, 22: 0.091, 24: 0.091, 29: 0.091, 13: 0.083, 14: 0.083, 20: 0.083, 28: 0.083, 6: 0.077, 8: 0.077, 0: 0.0}]


In [206]:
for i in range(len(ranking)):
  print(f'Query {i+1}: {queries[i+1]}\n')
  for j in range(len(ranking[i])):
    documento = list(ranking[i].keys())[j] #Para obtener los indices para buscar los documentos
    similitud = list(ranking[i].values())[j]
    print(f'Similitud: {similitud} | Documento {documento + 1}:{docs[documento]}')
  print('------------------------------------------------------------')



Query 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios

Similitud: 0.6 | Documento 2:Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
Similitud: 0.5 | Documento 7:La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
Similitud: 0.462 | Documento 11:Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.
Similitud: 0.417 | Documento 14:Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
Similitud: 0.385 | Documento 19:Factores de riesgo y estrategias para prevenir el agotamiento académico en estudiantes universitarios.
Similitud: 0.375 | Documento 13:Estrategias para mejorar la salud mental en estudiantes universitarios: el rol del apoyo psicológico y los grupos de estudio en el campus.
Similitud: 0.333 | Documento 23:Cómo la carga de trabajo académico af